In [1]:
# === WHAT YOU'LL NEED TO INTERACT WITH ===

# download_dir
# This defines the download location. The default, "/Users/ashlanjo/Downloads/Statcom_final", should be updated to a path
# appropriate for your system. It's recommended to create a dedicated folder for these downloads.
# The code will redownload files to this folder if they are stored elsewhere or don't follow the naming convention.
# To preserve old files, copy them into the new folder (rather than moving or renaming them).

# last_scrap_date
# This marks the last date the script is assumed to have been run. Files dated before this are skipped, saving significant time.
# It's recommended to backdate last_scrap_date by ~2 days to avoid missing late-uploaded files (e.g., uploads made later the same day).
# Files already downloaded won't be downloaded again, so this buffer is harmless.
# To force downloading older files, set last_scrap_date to a date far in the past (e.g., 20 years ago).
# This is easier than modifying the logic that skips older files.

# urls_to_run = sub_urls
# This should generally be left unchanged. However, to run the code for specific agencies, you can redefine sub_urls to a subset.
# Use print(urls_to_run) and check the links to ensure the correct agencies are selected.
# The order in sub_urls matches the order on the website.
# Be sure to reset it back to the full list once you're done.

# Manual interruptions and errors
# If you manually stop the code, make sure to run driver.quit() before doing anything else.
# This ensures that the browser is properly closed so it can reopen for future scraping sessions.
# If the code errors out, driver.quit() is automatically executed.

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

chrome_server = Service(executable_path='/Users/stefaneng/lib/chromedriver-mac-arm64/chromedriver')
chrome_server.start()
driver = webdriver.Chrome(service=chrome_server)
driver.get('http://www.google.com/')
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element("name", "q")
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

In [ ]:
#To Do
#Send to Kevin

In [10]:
# Selenium is used for web automation and scraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
# Time utilities for delays and timestamp handling
import time
from time import sleep
# Date and time parsing
from datetime import datetime
# File and path operations
from pathlib import Path
import os
import glob

In [11]:
#This code collects the links for all of the individual agencies (location where children stay)
#These links will be iterated through in future code chunks

# Initialize a Chrome browser session and navigate to the main licensing search page
chrome_server = Service(executable_path='/Users/stefaneng/lib/chromedriver-mac-arm64/chromedriver')
chrome_server.start()
driver = webdriver.Chrome(service=chrome_server)

try:
    driver.get("https://michildwelfarepubliclicensingsearch.michigan.gov/licagencysrch/")
    time.sleep(20) # Wait for the page to fully load (longer than usual due to dynamic content)
    # Prepare an empty list to store all agency-specific URLs found across pages

    sub_urls = []

    while True:
        # Locate all agency links on the current page using their XPATH
        agency_urls = driver.find_elements(By.XPATH, "//td/lightning-primitive-cell-factory/span/div/lightning-formatted-url/a")

        for agency_url in agency_urls:
            # Extract the actual hyperlink from each agency link element
            sub_url = agency_url.get_attribute('href')
            sub_urls.append(sub_url) # Add it to the growing list of sub-URLs

        try:
            # Try to locate and click the "Next" page button to load the next page of results
            next_button = driver.find_element(By.XPATH, "//lightning-button-icon[3]/button/lightning-primitive-icon")
            next_button.click()
        except ElementClickInterceptedException:
            # If the click fails (e.g., no more pages or overlay blocking it), stop the loop
            print("No more pages available.")
            break

finally:
    driver.quit()
# Show the number of agency URLs collected
len(sub_urls)

No more pages available.


284

In [ ]:
#All files prior to this date will not be downloaded
#All files on or after this data will usually be downloaded
#The exception is if the file is already in the download location with the correct name
last_scrap_date = "7-01-2025"
last_scrap_date = datetime.strptime(last_scrap_date, '%m-%d-%Y').date()

#All files will be downloaded here
download_dir = "/Users/stefaneng/Library/CloudStorage/Dropbox-UniversityofMichigan/Stefan Eng/MCYJ_parsing/test_download"

#This line can be adjusted to run only certain agencies
urls_to_run = sub_urls

In [14]:
#Initialize the browser
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_dir}
chromeOptions.add_experimental_option("prefs", prefs)
#driver = webdriver.Chrome(options=chromeOptions)
driver = webdriver.Chrome(service=chrome_server, options=chromeOptions)

#Iterates through each of the urls collected above
try:
    for url in urls_to_run:
            # Open the target page
        driver.get(url)
        time.sleep(20)

        # Get the document agency name
        document_agency_element = driver.find_elements(By.XPATH, "//lightning-layout-item[1]/slot/div[1]/div")
        document_agency = "_".join([element.text.strip().replace(" ", "_").replace("/", "_") for element in document_agency_element]) if document_agency_element else "Unknown_Agency"

        print(document_agency)

        # Locate all rows within the table
        table_rows = driver.find_elements(By.XPATH, "//lightning-datatable/div[2]/div/div/table/tbody/tr")
        print("Number of rows found:", len(table_rows))

        # Iterate through each row/ document
        for row in reversed(table_rows):
            try:
                # Extract document date from the 1st column and reformats it
                document_date_element = row.find_element(By.XPATH, "./td[1]")
                document_date = document_date_element.text.strip().replace(" ", "_")
                sanitized_date = document_date.replace("/", "-")
                sanitized_date = datetime.strptime(sanitized_date, '%m-%d-%Y').date()

                # Extract document name from the 2nd column and reformats it
                document_name_element = row.find_element(By.XPATH, "./td[2]")
                document_name_temp = document_name_element.text.strip().replace(" ", "_")
                document_name = document_name_temp.replace("/", "-")

                #Skips all the files that should have been scrapped on a prior run
                if sanitized_date < last_scrap_date:
                    print(document_name, " and all files before it should have been downloaded before")
                    break

                # Construct the new file name
                new_file_name = f"{document_agency}_{document_name}_{sanitized_date}.pdf"
                new_file_path = os.path.join(download_dir, new_file_name)

                #If the file is already downloaded (and follows the naming convention)
                #It will skip it
                if os.path.exists(new_file_path):
                    print(document_name, "has been downloaded since last scrap")
                    continue

                # Locate the clickable element in the 4th column
                clickable_element = row.find_element(By.XPATH, "./td[4]")

                # Check if the element is clickable
                if clickable_element.is_enabled() and clickable_element.is_displayed():
                    # Click to download
                    clickable_element.click()
                    print(f"Clicked on element in row with document '{document_name}'")

                    # Wait for the file to download (adjust wait time as needed)
                    time.sleep(10)

                    # Find the most recently downloaded file
                    list_of_files = glob.glob(f"{download_dir}/*")
                    latest_file = max(list_of_files, key=os.path.getctime)

                    # Rename the file
                    os.rename(latest_file, new_file_path)
                    print(f"Renamed file to: {new_file_name}")

            except Exception as e:
                print("Failed to click on element in row:", e)
    driver.quit()
finally:
    driver.quit()

Glen's_House
Number of rows found: 4
Glens_House_interim_2024  and all files before it should have been downloaded before
Mission_Ranch
Number of rows found: 11
Mission_Ranch_Renewal_2025  and all files before it should have been downloaded before
Heartland_Center_for_Autism
Number of rows found: 6
2025SIC0000409  and all files before it should have been downloaded before
Osceola_Youth_Center
Number of rows found: 25
Clicked on element in row with document '2025SIC0000657'
Renamed file to: Osceola_Youth_Center_2025SIC0000657_2025-07-09.pdf
Clicked on element in row with document '2025SIC000737'
Renamed file to: Osceola_Youth_Center_2025SIC000737_2025-07-09.pdf
Clicked on element in row with document '2025SIC0000763'
Renamed file to: Osceola_Youth_Center_2025SIC0000763_2025-07-02.pdf
2025SIC0000655  and all files before it should have been downloaded before
Let's_Talk_About_it_Girl's_Home_III
Number of rows found: 17
2025SIC0000512  and all files before it should have been downloaded be

KeyboardInterrupt: 